# Developing SQLAlchemy methods

### Methods to Build:

- Setup: Connecting to the Database
- Create a Table for Pump Data
- Insert Data into the Table
- Query Data from the Table
- Update Data in the Table
- Delete Data from the Table
- Use SQLAlchemy ORM to Define Models and Perform CRUD Operations

In [6]:
import os
from typing import Optional
from sqlalchemy import create_engine, Engine
from sqlalchemy import text

In [2]:

def create_sql_alchemy_engine(
    user: Optional[str] = None,
    password: Optional[str] = None,
    host: Optional[str] = "localhost",
    port: Optional[int] = 5432,  # PostgreSQL default port
    dbname: Optional[str] = None,
) -> Engine:
    """
    Create a SQLAlchemy engine for connecting to a PostgreSQL database.

    Parameters:
    user (str): Database username. If not provided, will use the 'DATABASE_USER' environment variable.
    password (str): Database password. If not provided, will use the 'DATABASE_PASSWORD' environment variable.
    host (str): Database host. Defaults to 'localhost'.
    port (int): Database port. Defaults to 5432.
    dbname (str): Database name. If not provided, will use the 'DATABASE_NAME' environment variable.

    Returns:
    Engine: A SQLAlchemy Engine instance for database operations.
    """
    # Retrieve from environment variables if not provided
    user = user or os.getenv("DATABASE_USER")
    password = password or os.getenv("DATABASE_PASSWORD")
    host = host or os.getenv("DATABASE_HOST", "localhost")
    port = port or int(os.getenv("DATABASE_PORT", 5432))  # Default to PostgreSQL port 5432
    dbname = dbname or os.getenv("DATABASE_NAME")

    if not user or not password or not dbname:
        raise ValueError("Database credentials and name must be provided.")

    # Construct the connection string
    connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}"

    # Create the SQLAlchemy engine
    return create_engine(connection_string)

In [7]:
# Example usage:
engine = create_sql_alchemy_engine(
    user='my_user',
    password='my_secrets',
    host='localhost',
    port=5432,
    dbname='data_generator_v1'
)

In [8]:
# Test the connection
with engine.connect() as connection:
    # Use the `text()` function for executing raw SQL
    result = connection.execute(text("SELECT 1"))
    # text("SELECT 1"): This creates a TextClause object, which is an Executable object in SQLAlchemy.
    # It allows you to execute raw SQL commands properly.
    print(result.fetchone())  # Should print (1,)

(1,)


### Understanding the Syntax

### 1. with engine.connect() as connection:
Purpose: This line is using a context manager (with statement) to create a new database connection from the SQLAlchemy engine.

engine.connect():

This method is used to create a new Connection object. The Connection object represents an active database connection. It provides a way to execute SQL statements, manage transactions, and interact with the database.
When you call engine.connect(), SQLAlchemy establishes a connection to the database from the connection pool maintained by the engine.
with Statement (Context Manager):

The with statement ensures that resources are properly managed. When the code block inside the with statement is done executing, it automatically releases the connection back to the pool (or closes it if it's no longer needed). This prevents resource leaks and ensures efficient use of connections.
It also handles any exceptions that might occur within the block, ensuring the connection is properly closed even if an error occurs.
Benefit of Using with:

Automatic cleanup: You don’t need to manually close the connection. It’s done automatically when the block is exited, either after successful execution or an error.
Reduces boilerplate code: You don’t need to write explicit try/finally blocks to ensure cleanup.

### 2. connection.execute()
Purpose: This method is used to execute a SQL statement on the database.

How It Works:

connection.execute(...) takes an Executable object (like text()), a SQL expression, or a SQLAlchemy statement object (e.g., select(), insert(), update(), delete()), and sends it to the database for execution.
Why Use .execute()?:

It abstracts the complexity of sending SQL commands to the database, making it easier to work with different database backends (e.g., PostgreSQL, MySQL, SQLite) without needing to change your code.
.execute() is a powerful function that supports a wide range of SQLAlchemy constructs, making it versatile for both raw SQL execution and ORM-based queries.

### 3. .fetchone()
Purpose: This method fetches a single row from the result set of the executed SQL query.

How It Works:

When you execute a SQL query that returns data (like SELECT), the execute() method returns a Result object.
Calling .fetchone() on the Result object retrieves the next row of the result set as a tuple.
If there are no more rows available, .fetchone() returns None.
Why Use .fetchone()?:

Efficient Memory Usage: If you only need one row from the result set, .fetchone() is more memory-efficient than .fetchall(), which retrieves all rows at once.
Useful for Single Row Queries: If you know your query is designed to return only one row (e.g., SELECT 1), .fetchone() is appropriate.

### Summary
- engine.connect(): Establishes a connection to the database.
- with ... as ...:: A context manager to handle resource cleanup automatically.
- connection.execute(...): Executes a SQL statement or SQLAlchemy expression.
- fetchone(): Retrieves the next row from the result set of the executed SQL statement.

In [9]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime, Engine
from sqlalchemy.types import Float as SQLAlchemyFloat, DateTime as SQLAlchemyDateTime


def map_dtype_to_sqlalchemy(dtype: str):
    """
    Map pandas DataFrame dtype to SQLAlchemy data type.
    """
    if pd.api.types.is_float_dtype(dtype):
        return SQLAlchemyFloat
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return SQLAlchemyDateTime
    # Add more mappings as needed (Integer, String, etc.)
    else:
        raise ValueError(f"Unsupported dtype: {dtype}")


def create_table_from_dataframe(df: pd.DataFrame, table_name: str, engine: Engine) -> None:
    """
    Create a table in the database from a pandas DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data structure.
    table_name (str): The name of the table to be created in the database.
    engine (Engine): An SQLAlchemy Engine instance connected to the database.
    """
    # Define metadata
    metadata = MetaData()

    # Define the table schema dynamically based on DataFrame columns and dtypes
    columns = [Column('id', Integer, primary_key=True)]  # Add an 'id' primary key column

    # Loop through DataFrame columns and create SQLAlchemy columns
    for column_name, dtype in df.dtypes.items():
        sqlalchemy_type = map_dtype_to_sqlalchemy(dtype)
        columns.append(Column(column_name, sqlalchemy_type))

    # Create the table schema
    table = Table(table_name, metadata, *columns)

    # Create the table in the database
    metadata.create_all(engine)
    print(f"Table '{table_name}' created from DataFrame.")



In [ ]:
# # Example usage
# if __name__ == "__main__":
#     # Sample DataFrame
#     data = {
#         'Temperature_C': [22.5, 23.1, 22.8],
#         'Pressure_MPa': [0.8, 0.85, 0.82],
#         'Vibration_mm_s': [1.2, 1.3, 1.1],
#         'Flow_Rate_l_min': [30.5, 31.0, 30.0],
#         'Humidity_%': [45.0, 46.0, 44.5]
#     }
#     df = pd.DataFrame(data)
#     df.index = pd.date_range(start='2024-08-23', periods=len(df), freq='H')

#     # Create an engine using environment variables or specified parameters
#     engine = create_sql_alchemy_engine(
#         user='my_user',
#         password='my_secrets',
#         host='localhost',
#         port=5432,
#         dbname='data_generator_v1'
#     )

#     # Create the table from the DataFrame
#     create_table_from_dataframe(df, "sensor_data", engine)


# theoretical stuff

### The Engine in SQLAlchemy is a core object that represents the interface to the database. Here’s a breakdown of what the Engine does:

- Connection Pooling: The Engine manages a pool of database connections. When you execute a query, the Engine provides a connection from this pool, making it efficient to execute multiple queries without needing to establish a new connection each time.

- Database Dialect: The Engine is configured with a dialect that is specific to the type of database you're using (PostgreSQL in this case). This dialect translates SQLAlchemy commands into the appropriate SQL for your database system.

- Execution Context: The Engine provides the execution context for SQL queries. It takes SQL expressions and translates them into the SQL string that is sent to the database.

- Thread-Safe: The Engine is designed to be shared among multiple threads, and it's safe to use concurrently. This is especially useful for web applications where multiple requests need to interact with the database.